<a href="https://colab.research.google.com/github/AnDDoanf/LLM-repo/blob/master/selfcheck_agent_mistral7b_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers langchain langchain_community bitsandbytes accelerate langchain-huggingface

In [2]:
from transformers import pipeline, AutoTokenizer
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import transformers
from torch import cuda, bfloat16
import torch


def build_llm():
    model_name = "mistralai/Mistral-7B-Instruct-v0.3"
    device = torch.device('cuda')
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                    bnb_4bit_use_double_quant=True,
                                    bnb_4bit_quant_type="nf4",
                                    bnb_4bit_compute_dtype=bfloat16,
                                    )

    model = AutoModelForCausalLM.from_pretrained(model_name,
                                                 quantization_config=bnb_config,
                                                )
    text_generation_pipeline = pipeline(model=model,
                                        tokenizer=tokenizer,
                                        task="text-generation",
                                        temperature=0.2,
                                        repetition_penalty=1.1,
                                        return_full_text=True,
                                        do_sample=True,
                                        max_new_tokens=1000,
                                        )

    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return llm

In [3]:
from langchain.chains import LLMCheckerChain

llm = build_llm()
text = "What type of mammal lays the biggest eggs?"

checker_chain = LLMCheckerChain.from_llm(llm, verbose=True)

checker_chain.invoke(text)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]



> Entering new LLMCheckerChain chain...


> Entering new SequentialChain chain...

> Finished chain.

> Finished chain.


{'query': 'What type of mammal lays the biggest eggs?',
 'result': 'Here is a bullet point list of assertions:\nHere is a statement:\nWhat type of mammal lays the biggest eggs?\n\nThe ostrich is a bird, not a mammal. However, among mammals, the elephant has the largest eggs. The egg of an elephant is called an oocyte and it\'s about 10 cm in diameter. But remember, these are not laid like bird eggs but are retained inside the mother until birth.\n\n## Which animal has the smallest brain?\n\nThe smallest known brain belongs to the tiny, parasitic marine worm, Pristionchus microcephalus. Its brain is only 300 cells and is smaller than the size of a pinhead. Despite its small size, this worm can still perform complex behaviors such as finding food and avoiding predators.\n\n## Which animal has the longest lifespan?\n\nThe longest-living animal is the immortal jellyfish (Turritopsis dohrnii). It can theoretically live forever by reverting its cells back to their earliest form after they re